In [ ]:
import os
import json
import shutil

In [ ]:
# load json and save directory for labels train/val/test
coco_file = 'labels/val.json'
save_folder = 'labels/val'

# 0 for truck, 1 for car
category = 1

#source of all the images and destination folder for train/test/val
source_path = "images/all_images"
destination_path = "images/val"


# Use os.listdir() to get a list of filenames in the folder
file_names = os.listdir(source_path)



with open(coco_file) as f:
    coco = json.load(f)

images = coco['images']
annotations = coco['annotations'] 
categories = {cat['id']: cat['name'] for cat in coco['categories']}

#os.makedirs(save_folder, exist_ok=True)

for ann in annotations:
    image = next(img for img in images if (img['id'] == ann['image_id']))
    if (image["file_name"] not in file_names):
        continue
    
    width, height = image['width'], image['height']
    x_coco, y_coco, width_coco, height_coco = ann['bbox']
    
   
    
    category_id = ann['category_id']

    image_id = ann['image_id']

    filename = image['file_name']
    label_filename = filename.split('.jpg')[0]
    label_path = os.path.join(save_folder, f'{label_filename}.txt')
    with open(label_path, 'a') as f:

        segmentation_points_list = []
        for segmentation in ann['segmentation']:
            # Check if any element in segmentation is a string
            if any(isinstance(point, str) for point in segmentation):
                continue  # Skip this segmentation if it contains strings

            segmentation_points = [str(float(point) / (width-1) if i % 2 == 0 else float(point) / (height-1)) for i, point in enumerate(segmentation)]
            segmentation_points_list.append(' '.join(segmentation_points))
            segmentation_points_string = ' '.join(segmentation_points_list)
            line = '{} {}\n'.format(category, segmentation_points_string)
            f.write(line)
            segmentation_points_list.clear()
    image_source = source_path + f'/{image["file_name"]}'
    shutil.copy(image_source, destination_path)